# A Simple Boolean Retrieval System

In [100]:
from functools import total_ordering, reduce  # not essential but reduces the code we have to write
import csv     # for csv files
import re      # for regular expressions
import pickle  # to save the index

## Postings

A `Posting` object is simply the docID of a document. It has a method `get_from_corpus` that given the corpus retrieves the document corresponding to that docID. Then it has some comparison methods to check if two docID are equal, one greater than the other, etc.

In [2]:
@total_ordering   # takes a class where we have defined at least the methods `eq` and `gt`/`lt` and defines in a consistent way all the other methods (otherwise we should implement them all by hand)
class Posting:
    
    def __init__(self, docID):
        """ Class constructor.
        """
        self._docID = docID
        
    def get_from_corpus(self, corpus):  # return from the corpus the doc corresponding to that docID. In the list you only save the docID, not the all document
        """ Returns the document corresponding to that docID from the corpus.
        """
        return corpus[self._docID]
    
    def __eq__(self, other: 'Posting'):  # euqality comparator
        """ Performs the comparison between this posting and another one.
        Since the ordering of the postings is only given by their docID,
        they are equal when their docIDs are equal.
        """
        return self._docID == other._docID
    
    def __gt__(self, other: 'Posting'):  # greather than comparator
        """ As in the case of __eq__, the ordering of postings is given
        by the ordering of their docIDs.
        """
        return self._docID > other._docID
    
    def __repr__(self):       # for debagging purposes to print the class
        """ String representation of the class.
        """
        return str(self._docID)

## Posting Lists

A `PostingList` object is a list of `Posting`s. You can construct an empty `PostingList` with `__init__`, or construct and initialize a `PostingList` directly with one docID with `from_docID`, or you can create a `PostingList` object with an already existing list using `from_posting_list`. Then you can merge two posting list with `merge` (the one in input will be added at the end of the one on which the mehod `merge` is called, without any checking on the total ordering of the list), you can intersect them with `intersection` or you can unify them with `union`. With `get_from_corpus` we can retrieve the documents corresponding to the docID stored in this `PostingList`.

In [3]:
class PostingList:
    
    def __init__(self):
        """ Class constructor.
        """
        self._postings = []    # it has as an attribute a list of posting
        
    @classmethod     # to define another constructor. It will return another PostingList like a constructor
    def from_docID(cls, docID):
        """ A posting list can be constructed starting from a single docID.
        """
        plist = cls()
        plist._postings = [(Posting(docID))]
        return plist
    
    @classmethod
    def from_posting_list(cls, postingList: 'PostingList'):
        """ A posting list can also be constructed by using another posting list.
        """
        plist = cls()
        plist._postings = postingList   # we use it as the postins of this PostingList
        return plist
    
    def merge(self, other: 'PostingList'):  # we have to merge postinglists
        """ Merges the other posting list to this one in a desctructive
        way, i.e., modifying the current posting list. This method assumes
        that all the docIDs of the second list are higher than the ones
        in this list. It assumes the two posting lists to be ordered
        and non-empty. Under those assumptions duplicate docIDs are
        discarded.
        """
        i = 0
        last = self._postings[-1]   # the self eleemnt of the current postinglist
        while (i < len(other._postings) and last == other._postings[i]):  # we can have the same docID multiple times and when e merge them we don't want them multiple times
            i += 1
        self._postings += other._postings[i:]
        
    def intersection(self, other: 'PostingList'):
        """ Returns a new posting list resulting from the intersection
        of this one and the one passed as argument.
        """
        intersection = []
        i = 0
        j = 0
        while (i < len(self._postings) and j < len(other._postings)):  # until we reach the end of a posting list
            if (self._postings[i] == other._postings[j]):
                intersection.append(self._postings[i])
                i += 1
                j += 1
            elif (self._postings[i] < other._postings[j]):
                i += 1
            else:
                j += 1
        return PostingList.from_posting_list(intersection)
    
    def union(self, other: 'PostingList'):
        """ Returns a new posting list resulting from the union of this
        one and the one passed as argument.
        """
        union = []
        i = 0
        j = 0
        while (i < len(self._postings) and j < len(other._postings)):
            if (self._postings[i] == other._postings[j]):
                union.append(self._postings[i])
                i += 1
                j += 1
            elif (self._postings[i] < other._postings[j]):
                union.append(self._postings[i])   # because i is the smallest one
                i += 1
            else:
                union.append(other._postings[j]) 
                j += 1
        for k in range(i, len(self._postings)):  # we have to append the remaining elements of the non emptied list
            union.append(self._postings[k])
        for k in range(j, len(other._postings)):
            union.append(other._postings[k])
        return PostingList.from_posting_list(union)
    
    def get_from_corpus(self, corpus):   # used when we have a posting list that is the result of a query, but I don't want the docID, I want the docs!
        return list(map(lambda x: x.get_from_corpus(corpus), self._postings))  # I return a list of documents
    
    def __repr__(self):
        return ", ".join(map(str, self._postings))

## Terms

A `Term` object contains both the word itself and the `PostingList` with all the docIDs of the documents in which the word is contained. The `merge` function merges the `PostingList`s of two equal `Term`s. Then we have some comparison methods to check if two `Term`s are equal or one is greater then the other, etc.

In [4]:
class ImpossibleMergeError(Exception):
    pass

@total_ordering  # to have all the ordering methods defined automatically
class Term:
    
    def __init__(self, term, docID):   # we create a term with a DocID, we sort them and we merge the equal terms
        self.term = term
        self.posting_list = PostingList.from_docID(docID)
        
    def merge(self, other: 'Term'):   # when we merge two terms
        """ Merges (destructively) this term and the corresponding posting list
        with another equal term and its corrsponding posting list.
        """
        if (self.term == other.term): # cannot merge posting lists with different terms!
            self.posting_list.merge(other.posting_list)  # merge the current posting list with the one of the other
        else: 
            raise ImpossibleMergeError # (some kind of error) error of impossible merge
            
    def __eq__(self, other: 'Term'):
        return self.term == other.term
    
    def __gt__(self, other: 'Term'):
        return self.term > other.term
    
    def __repr__(self):
        return self.term + ": " + repr(self.posting_list)

## Inverted Index

In [5]:
# We have to do some step of tokenization and normalization

def normalize(text):
    """ A simple funzion to normalize a text.
    It removes everything that is not a word, a space or an hyphen
    and downcases all the text.
    """
    no_punctuation = re.sub(r'[^\w^\s^-]', '', text)  # the text that matches a certain pattern will be substittuted with the second expression. ^\w → not something alphanumeric, ^\s → not some space, ^- → not a dash, replace it with '', the empty string
    downcase = no_punctuation.lower()  # put everything to lower case
    return downcase

def tokenize(movie: 'MovieDescription'):
    """ Returns a list, which is a posting list, from a movie
    description of all tokens present in the description.
    """
    text = normalize(movie.description)
    return list(text.split())

Function to print a progress bar, taken from [here](https://stackoverflow.com/questions/3160699/python-progress-bar).

In [6]:
import time, sys

def update_progress(progress):
    """ Displays or updates a console progress bar.
    Accepts a float between 0 and 1. Any int will be converted to a float.
    A value under 0 represents a 'halt'.
    A value at 1 or bigger represents 100%
    """
    barLength = 40 # Modify this to change the length of the progress bar
    status = ""
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
        status = "error: progress var must be float\r\n"
    if progress < 0:
        progress = 0
        status = "Halt...\r\n"
    if progress >= 1:
        progress = 1
        status = "Done!\r\n"
    block = int(round(barLength*progress))
    text = "\r[{0}] {1}% {2}".format( "#"*block + "."*(barLength-block), round(progress*100, 2), status)
    sys.stdout.write(text)
    sys.stdout.flush()

An `InvertedIndex` object contains a dictionary with as keys the words and as values the `Term` associated to that word, which, we reall, contains the `PostingList` associated to the word.

In [7]:
class InvertedIndex:
    
    def __init__(self):
        self._dictionary = {}  # initially the inverted index dictionary is empty
        
    @classmethod  # instead of having this method associated to a specific instance/object of the class InvertedIndex we write InvertedIndex.from_corpus(). Because you can have only one __init__ method, so you use @classmethod to have multiple constructors. It's like a static method in Java
    def from_corpus(cls, corpus: list):
        intermediate_dict = {}   # we cheat a little bit and use a Python dictionary → we should create a big list, sort it and merge everything
        print("Processing the corpus to create the index...")
        for docID, document in enumerate(corpus): # NB: corpus: collection (list) of objects of type MovieDescription
            tokens = tokenize(document) # document is a MovieDescription object
            for token in tokens:
                term = Term(token, docID)
                try:
                    intermediate_dict[token].merge(term)  # I merge the two posting lists → Term.merge() which calls PostingList.merge()
                except KeyError:
                    intermediate_dict[token] = term # for when the term is not present in the dict
            update_progress(docID/len(corpus))  # to see the progressing of our indexing
                
        idx = cls()  # we call the constructor of the class = InvertedIndex
        idx._dictionary = sorted(intermediate_dict.values())  # list of all the sorted terms
        return idx
    
    def __getitem__(self, key): # indexing the inverted index using as keys the terms
        for term in self._dictionary:  # we could do a binary search
            if term.term == key:
                return term.posting_list  # quering the index with a  word returns the PostingList associated to that word
        raise KeyError(f"The term {key} is not present in the index.") # the key is not present!
        
    def __repr__(self):
        return "A dictionary with " + str(len(self._dictionary)) + " terms"

## Reading the Corpus

A `MovieDescription` object has a title and a description.

In [8]:
@total_ordering
class MovieDescription:  # container for all the info we have about the movie
    
    def __init__(self, title, description):
        self.title = title
        self.description = description
        
    def __eq__(self, other: 'MovieDescription'):
        return self.title == other.title
    
    def __gt__(self, other: 'MovieDescription'):
        return self.title > other.title
        
    def __repr__(self):
        return self.title

In [9]:
def read_movie_descriptions():
    filename = 'data/plot_summaries.txt'   # not very portable but done for the sake of simplicity
    movie_names_file = 'data/movie.metadata.tsv'
    with open(movie_names_file, 'r') as csv_file:
        movie_names = csv.reader(csv_file, delimiter = '\t')   # we define the csv reader
        names_table = {}   # Python dictionary with all the names of the films: key = movieID, value = movie title
        for name in movie_names:
            names_table[name[0]] = name[2] # the first element is the ID, the third elemnt is the title
    # Now we have all the associations between ID and title, we miss the move description

    with open(filename, 'r') as csv_file:
        descriptions = csv.reader(csv_file, delimiter = '\t')
        corpus = []   # collection (list) of objects of type MovieDescription
        for desc in descriptions:
            try:      # at least in this dataset there are some errors so some descriptions have not a matching ID
                movie = MovieDescription(names_table[desc[0]], desc[1]) # the first element is the ID, the second the description
                corpus.append(movie)
            except KeyError:  # in case we don't find the title associated to that ID
                pass
        return corpus

## IR System

An `IRsystem` object contains the entire corpus and the `InvertedIndex`.

In [10]:
class IRsystem:
    
    def __init__(self, corpus, index):
        self._corpus = corpus
        self._index = index
        
    @classmethod
    def from_corpus(cls, corpus): # generate the entire inverted index calling the constructor
        index = InvertedIndex.from_corpus(corpus)
        return cls(corpus, index)  # retrun the constructor when we have yet the index
    
    def answer_and_query(self, words):  # For now only queries with AND. We have a list of words like ['cat', 'batman']
        norm_words = map(normalize, words)  # Normalize all the words. IMPORTANT!!! If the user uses upper-case we will not have ANY match! We have to perform the same normalization of the docs in the corpus on the query!
        postings = map(lambda w: self._index[w], norm_words) # get the posting list for each word → list of posting lists
        plist = reduce(lambda x, y: x.intersection(y), postings)  # apply the function to the two items of the list, then apply it to the result with the third, then the result with the fourt term and so on until the end of the list
        return plist.get_from_corpus(self._corpus) # retrun the documents
    
    def answer_or_query(self, words):
        norm_words = map(normalize, words)
        postings = map(lambda w: self._index[w], norm_words)
        plist = reduce(lambda x, y: x.union(y), postings)
        return plist.get_from_corpus(self._corpus)

In [11]:
def and_query(ir, text, noprint=True):
    words = text.split()
    answer = ir.answer_and_query(words)  # list of documents
    if not noprint:
        for movie in answer:
            print(movie)
    return answer
        
def or_query(ir, text, noprint=True):
    words = text.split()
    answer = ir.answer_or_query(words)
    if not noprint:
        for movie in answer:
            print(movie)
    return answer

## Examples

In [12]:
PostingList.from_docID(10)   # creates a PostingL with the docID 10

10

In [13]:
x = Term("cat", 3)
y = Term("cat", 6)
x.merge(y)
print(x)

cat: 3, 6


In [15]:
x = Posting(1)
y = Posting(2)
print(x < y)
print(x <= y)

True
True


In [16]:
normalize("e.g., this is A TeSt") # we remove punctuation and put everything to lowercase

'eg this is a test'

In [17]:
tokenize(MovieDescription("Title", "This is a movie Description."))

['this', 'is', 'a', 'movie', 'description']

In [18]:
corpus = read_movie_descriptions()

In [20]:
print(idx)

A dictionary with 194757 terms


In [21]:
idx['batman']  # all the docIDs containing 'batman' in the description!

334, 2990, 3463, 3519, 3545, 5510, 6854, 7105, 7358, 8467, 9503, 10360, 10727, 10933, 12458, 12492, 12967, 13095, 14199, 17366, 18875, 19381, 19675, 20598, 20808, 21070, 22147, 24393, 24484, 24658, 25866, 30601, 31272, 31508, 33213, 33638, 35356, 35980, 37238, 37389, 38152, 39092, 40499, 40596, 40821

In [22]:
ir = IRsystem(corpus, idx)

In [23]:
and_query(ir, "frodo Gandalf")

[The Lord of the Rings: The Fellowship of the Ring,
 The Lord of the Rings,
 The Hunt for Gollum,
 The Return of the King,
 Date Movie,
 The Lord of the Rings: The Two Towers,
 The Lord of the Rings: The Return of the King]

In [98]:
try:
    and_query(ir, "thig")
except KeyError:
    print(sys.exc_info()[1])

'ciao'


In [24]:
and_query(ir, "yoda luke darth")

[Star Wars Episode V: The Empire Strikes Back,
 Something, Something, Something Dark Side,
 Return of the Ewok,
 Star Wars Episode III: Revenge of the Sith,
 Star Wars Episode VI: Return of the Jedi,
 It's a Trap!]

In [25]:
or_query(ir, "frodo yoda")

[Star Wars Episode V: The Empire Strikes Back,
 Star Wars Episode II: Attack of the Clones,
 George Lucas in Love,
 The Lord of the Rings: The Fellowship of the Ring,
 The Lord of the Rings,
 Something, Something, Something Dark Side,
 The Hunt for Gollum,
 The Return of the King,
 Return of the Ewok,
 Aliens in the Wild, Wild West,
 Star Wars Episode III: Revenge of the Sith,
 Star Wars Episode VI: Return of the Jedi,
 Star Wars: The Clone Wars,
 Date Movie,
 Gulliver's Travels,
 Lego Star Wars: The Quest for R2-D2,
 The Lord of the Rings: The Two Towers,
 It's a Trap!,
 The Lord of the Rings: The Return of the King,
 LEGO Star Wars: Revenge of the Brick]

In [26]:
frodo_query = and_query(ir, "frodo")
yoda_query = and_query(ir, "yoda")
frodo_or_yoda_query = or_query(ir, "frodo yoda")
# frodo_query.extend(yoda_query)  # then print 'frodo_query' !!!!
frodo_p_yoda_query = frodo_query + yoda_query
assert sorted(frodo_p_yoda_query) == sorted(frodo_or_yoda_query)

In [27]:
def make_unique(original_list):
    unique_list = []
    [unique_list.append(obj) for obj in original_list if obj not in unique_list]
    return unique_list

In [81]:
frodo_query = and_query(ir, "frodo")
yoda_query = and_query(ir, "yoda")
third_query = and_query(ir, "gandalf")
my_or_query = sorted(or_query(ir, "frodo yoda gandalf"))
all_query = sorted(frodo_query + yoda_query + third_query)

#all_query = make_unique(all_query)
print(len(frodo_query), len(yoda_query), len(third_query))
print(len(all_query), len(make_unique(all_query)), len(my_or_query))

print(sorted(frodo_query))
print("-----------")
print(sorted(yoda_query))
print("-----------")
print(sorted(third_query))
print("-----------")
print(sorted(my_or_query))

7 13 8
28 21 21
[Date Movie, The Hunt for Gollum, The Lord of the Rings, The Lord of the Rings: The Fellowship of the Ring, The Lord of the Rings: The Return of the King, The Lord of the Rings: The Two Towers, The Return of the King]
-----------
[Aliens in the Wild, Wild West, George Lucas in Love, Gulliver's Travels, It's a Trap!, LEGO Star Wars: Revenge of the Brick, Lego Star Wars: The Quest for R2-D2, Return of the Ewok, Something, Something, Something Dark Side, Star Wars Episode II: Attack of the Clones, Star Wars Episode III: Revenge of the Sith, Star Wars Episode V: The Empire Strikes Back, Star Wars Episode VI: Return of the Jedi, Star Wars: The Clone Wars]
-----------
[Date Movie, Imaginationland Episode II, The Hunt for Gollum, The Lord of the Rings, The Lord of the Rings: The Fellowship of the Ring, The Lord of the Rings: The Return of the King, The Lord of the Rings: The Two Towers, The Return of the King]
-----------
[Aliens in the Wild, Wild West, Date Movie, George Luca

In [99]:
frodo_query = and_query(ir, "frodo")
yoda_query = and_query(ir, "yoda")
third_query = and_query(ir, "love")
my_or_query = sorted(or_query(ir, "frodo yoda love"))
all_query = sorted(frodo_query + yoda_query + third_query)
#all_query = make_unique(all_query)

#print(sorted(frodo_query))
#print("-----------")
#print(sorted(yoda_query))
#print("-----------")
#print(sorted(catwoman_query))
#print("-----------")
#print(sorted(my_or_query))

print(len(frodo_query), len(yoda_query), len(third_query))
print(len(all_query), len(my_or_query))

all_query_uniq = make_unique(all_query)
my_or_query_unique = make_unique(my_or_query)
print(len(all_query_uniq), len(my_or_query_unique))

assert sorted(all_query_uniq) == sorted(my_or_query_unique)

7 13 10461
10481 10474
10175 10175


In [91]:
for i in range(len(all_query_uniq)):
    if all_query_uniq[i] != my_or_query_unique[i]:
        print(f"{all_query_uniq[i-1].title:<30}\t --  {my_or_query_unique[i-1]}")
        print(f"{all_query_uniq[i].title:<30}\t --  {my_or_query_unique[i]}")
        print(f"{all_query_uniq[i+1].title:<30}\t --  {my_or_query_unique[i+1]}")
        break

In [84]:
errors = {'love': 7, 'mother': 2, 'father': 7, 'cat': 1, 'me': 1}

missing_titles = []
for i in range(len(my_or_query)):
    if all_query[i] != my_or_query[i]:
        missing_titles.append(all_query[i])
        print(f"{all_query[i-1].title:<30}\t --  {my_or_query[i-1]}")
        print(f"{all_query[i].title:<30}\t --  {my_or_query[i]}")
        print(f"{all_query[i+1].title:<30}\t --  {my_or_query[i+1]}")
        break

print(f"\nSecond missing word:")
for j in range(i+1,len(my_or_query)):
    if all_query[j] != my_or_query[j-1]:
        missing_titles.append(all_query[j])
        print(f"{all_query[j-1].title:<30}\t --  {my_or_query[j-2]}")
        print(f"{all_query[j].title:<30}\t --  {my_or_query[j-1]}")
        print(f"{all_query[j+1].title:<30}\t --  {my_or_query[j]}")
        break

print(missing_titles)

Date Movie                    	 --  Date Movie
Date Movie                    	 --  Date Night
Date Night                    	 --  Daughter of the East

Second missing word:
Lego Star Wars: The Quest for R2-D2	 --  Lego Star Wars: The Quest for R2-D2
Lego Star Wars: The Quest for R2-D2	 --  Lekhayude Maranam Oru Flashback
Lekhayude Maranam Oru Flashback	 --  Lemonade Joe
[Date Movie, Lego Star Wars: The Quest for R2-D2]


In [77]:
for i, movie in enumerate(corpus):
    if movie == missing_titles[0]:
    # if movie.title == missing_titles[0].title:
        print(f"Found: {missing_titles[0].title} in {i} → {corpus[i]}")

print("\nmy_or_query")
for i, movie in enumerate(my_or_query):
    if movie == missing_titles[0]:
        print(f"Found: {missing_titles[0].title} in {i} → {my_or_query[i]}")

print("\nall_query")
for i, movie in enumerate(all_query):
    if movie == missing_titles[0]:
        print(f"Found: {missing_titles[0].title} in {i} → {all_query[i]}")
        
print("\nfrodo_query")
for i, movie in enumerate(frodo_query):
    if movie == missing_titles[0]:
        print(f"Found: {missing_titles[0].title} in {i} → {frodo_query[i]}")
        
print("\nyoda_query")
for i, movie in enumerate(yoda_query):
    if movie == missing_titles[0]:
        print(f"Found: {missing_titles[0].title} in {i} → {yoda_query[i]}")

print("\nthirs_query")
for i, movie in enumerate(third_query):
    if movie == missing_titles[0]:
        print(f"Found: {missing_titles[0].title} in {i} → {third_query[i]}")
        
print(len(make_unique(all_query)), len(make_unique(my_or_query)))

Found: Date Movie in 34599 → Date Movie

my_or_query
Found: Date Movie in 1413 → Date Movie

all_query
Found: Date Movie in 1413 → Date Movie
Found: Date Movie in 1414 → Date Movie

frodo_query
Found: Date Movie in 4 → Date Movie

yoda_query

thirs_query
Found: Date Movie in 5780 → Date Movie


In [75]:
print(len(corpus), len(make_unique(corpus)))

Found: Date Movie in 34599 → Date Movie

my_or_query
Found: Date Movie in 1413 → Date Movie

all_query
Found: Date Movie in 1413 → Date Movie
Found: Date Movie in 1414 → Date Movie
6933 6933
42204 39914
